In [33]:
# 필요한 라이브러리 불러오기
import torch
import os
import pandas as pd
import json
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

In [34]:
# 데이터셋 경로 설정: 각 경로에서 이미지 파일 목록을 가져옵니다.
training_image = os.listdir('./data/training_image')   # 훈련 이미지 경로에서 파일 목록 가져오기
validation_image = os.listdir('./data/validation_image')  # 검증 이미지 경로에서 파일 목록 가져오기

# 데이터셋 파일명에서 속성 정보를 분리하여 데이터프레임 생성
column_name = ['W/T', 'image_id', 'period', 'style', 'Sex']  # 파일명에서 추출할 열 이름

# 훈련 데이터셋 파일명과 속성 정보로 데이터프레임 생성
training_data = {'file_name': training_image}  # 훈련 이미지 파일명을 딕셔너리로 생성
training_df = pd.DataFrame(training_data)  # 딕셔너리를 데이터프레임으로 변환
# 파일명에서 확장자(.jpg)를 제거하고 '_' 기준으로 나눈 후, 지정한 column_name에 맞춰 할당
training_df[column_name] = training_df['file_name'].str.replace('.jpg', '').str.split('_', expand=True)

# 검증 데이터셋 파일명과 속성 정보로 데이터프레임 생성
validation_data = {'file_name': validation_image}  # 검증 이미지 파일명을 딕셔너리로 생성
validation_df = pd.DataFrame(validation_data)  # 딕셔너리를 데이터프레임으로 변환
# 파일명에서 확장자(.jpg)를 제거하고 '_' 기준으로 나눈 후, 지정한 column_name에 맞춰 할당
validation_df[column_name] = validation_df['file_name'].str.replace('.jpg', '').str.split('_', expand=True)

In [35]:
# 데이터셋 경로 설정: 각 경로에서 라벨 파일 목록을 가져옵니다.
training_label = os.listdir('./data/training_label') # 훈련 라벨 경로에서 파일 목록 가져오기
validation_label = os.listdir('./data/validation_label') # 검증 라벨 경로에서 파일 목록 가져오기

# 데이터셋 파일명에서 속성 정보를 분리하여 데이터프레임 생성
column_name_2 = ['W/T', 'image_id', 'period', 'style', 'Sex', 'survey_id'] # 파일명에서 추출할 열 이름

# 훈련 데이터셋 파일명과 속성 정보로 데이터프레임 생성
training_label_data = {'file_name': training_label} # 훈련 라벨 파일명을 딕셔너리로 생성
training_label_df = pd.DataFrame(training_label_data) # 딕셔너리를 데이터프레임으로 변환
# 파일명에서 확장자(.json)를 제거하고 '_' 기준으로 나눈 후, 지정한 column_name에 맞춰 할당
training_label_df[column_name_2] = training_label_df['file_name'].str.replace('.json', '').str.split('_', expand=True)

# 검증 데이터셋 파일명과 속성 정보로 데이터프레임 생성
validation_label_data = {'file_name': validation_label} # 검증 라벨 파일명을 딕셔너리로 생성
validation_label_df = pd.DataFrame(validation_label_data) # 딕셔너리를 데이터프레임으로 변환
# 파일명에서 확장자(.json)를 제거하고 '_' 기준으로 나눈 후, 지정한 column_name에 맞춰 할당
validation_label_df[column_name_2] = validation_label_df['file_name'].str.replace('.json', '').str.split('_', expand=True)

In [36]:
# training_label_df에서 training_df의 'image_id' 값과 일치하는 행만 필터링하여 새로운 데이터프레임 생성
drop_training_label_df = training_label_df[training_label_df['image_id'].isin(training_df['image_id'])]

# validation_label_df에서 validation_df의 'image_id' 값과 일치하는 행만 필터링하여 새로운 데이터프레임 생성
drop_validation_label_df = validation_label_df[validation_label_df['image_id'].isin(validation_df['image_id'])]

In [37]:
# drop_training_label_df 데이터프레임에 'T/V'라는 새로운 컬럼을 추가하고,
# 모든 행에 'Train' 값을 설정하여 해당 데이터가 훈련 데이터임을 표시
drop_training_label_df.loc[:, 'T/V'] = 'Train'

# drop_validation_label_df 데이터프레임에 'T/V'라는 새로운 컬럼을 추가하고,
# 모든 행에 'Validation' 값을 설정하여 해당 데이터가 검증 데이터임을 표시
drop_validation_label_df.loc[:, 'T/V'] = 'Validation'

In [38]:
# 훈련 및 검증 데이터의 JSON 파일 경로 지정
train_path = './data/training_label'
valid_path = './data/validation_label'

# JSON 파일을 읽어 데이터를 추출하는 함수를 정의
def extract_json_data(file_name):
    # JSON 파일을 열어서 데이터를 읽음
    with open(file_name, 'r') as f:
        data = json.load(f)
    
    # 필요한 데이터인 'user'의 'R_id'와 'item'의 'survey'에서 'Q5' 값을 추출
    R_id = data['user']['R_id']
    Q5 = data['item']['survey']['Q5']
    
    return R_id, Q5

# drop_training_label_df와 drop_validation_label_df에 R_id와 Q5 값을 담을 새로운 열을 생성하고 기본값을 None으로 설정
drop_training_label_df.loc[:, 'R_id'] = None
drop_training_label_df.loc[:, 'Q5'] = None
drop_validation_label_df.loc[:, 'R_id'] = None
drop_validation_label_df.loc[:, 'Q5'] = None

# drop_training_label_df의 각 행에 대해 JSON 파일에서 데이터를 추출하여 데이터프레임에 추가
for index, row in drop_training_label_df.iterrows():
    # 파일 이름을 전체 경로로 변환
    file_name = os.path.join(train_path, row['file_name'])
    
    # JSON 파일에서 R_id와 Q5 데이터를 추출
    R_id, Q5 = extract_json_data(file_name)
    
    # 추출한 R_id와 Q5 값을 해당 행의 'R_id'와 'Q5' 열에 추가
    drop_training_label_df.at[index, 'R_id'] = R_id
    drop_training_label_df.at[index, 'Q5'] = Q5

# drop_validation_label_df의 각 행에 대해 JSON 파일에서 데이터를 추출하여 데이터프레임에 추가
for index, row in drop_validation_label_df.iterrows():
    # 파일 이름을 전체 경로로 변환
    file_name = os.path.join(valid_path, row['file_name'])
    
    # JSON 파일에서 R_id와 Q5 데이터를 추출
    R_id, Q5 = extract_json_data(file_name)
    
    # 추출한 R_id와 Q5 값을 해당 행의 'R_id'와 'Q5' 열에 추가
    drop_validation_label_df.at[index, 'R_id'] = R_id
    drop_validation_label_df.at[index, 'Q5'] = Q5

In [39]:
# drop_training_label_df와 drop_validation_label_df 데이터프레임을 연결하여 하나의 데이터프레임으로 결합
# ignore_index=True로 설정하여 기존 인덱스를 무시하고 새로운 연속 인덱스를 생성
drop_df = pd.concat([drop_training_label_df, drop_validation_label_df], ignore_index=True)

In [40]:
# drop_df 데이터프레임에 새로운 열 'jpg_name'을 생성
# 'W/T', 'image_id', 'period', 'style', 'Sex' 열의 값들을 결합하여 파일명 형식으로 지정
# 각 값을 문자열로 변환하여 '_'로 연결하고, 마지막에 '.jpg'를 추가하여 파일명 생성
drop_df['jpg_name'] = drop_df['W/T'].astype(str) + '_' + \
                      drop_df['image_id'].astype(str) + '_' + \
                      drop_df['period'].astype(str) + '_' + \
                      drop_df['style'].astype(str) + '_' + \
                      drop_df['Sex'].astype(str) + '.jpg'

In [41]:
# Training과 Validation을 구분한 후 선호/비선호 구분
training_preference = drop_df[(drop_df['T/V'] == 'Train') & (drop_df['Q5'] == 2)]
training_non_preference = drop_df[(drop_df['T/V'] == 'Train') & (drop_df['Q5'] == 1)]

validation_preference = drop_df[(drop_df['T/V'] == 'Validation') & (drop_df['Q5'] == 2)]
validation_non_preference = drop_df[(drop_df['T/V'] == 'Validation') & (drop_df['Q5'] == 1)]


# 응답자 ID를 기준으로 그룹화하고 스타일 선호 및 비선호를 리스트로 수집
summary_df = pd.DataFrame({
    #'응답자 ID': drop_df['R_id'].unique(),
    'Training 스타일 선호': training_preference.groupby('R_id')['jpg_name'].apply(list),
    'Training 스타일 비선호': training_non_preference.groupby('R_id')['jpg_name'].apply(list),
    'Validation 스타일 선호': validation_preference.groupby('R_id')['jpg_name'].apply(list),
    'Validation 스타일 비선호': validation_non_preference.groupby('R_id')['jpg_name'].apply(list)
})

summary_df = summary_df.applymap(lambda x: x if isinstance(x, list) else [])

/tmp/ipykernel_82888/3443426144.py:18: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  summary_df = summary_df.applymap(lambda x: x if isinstance(x, list) else [])


In [42]:
# summary_df에서 필요한 데이터를 리스트로 변환하여 딕셔너리로 재구성
user_data = {
    "R_id": summary_df.index.tolist(),  # R_id는 index로 설정되어 있는 경우
    "Training_Like": summary_df["Training 스타일 선호"].tolist(),
    "Training_Dislike": summary_df["Training 스타일 비선호"].tolist(),
    "Validation_Like": summary_df["Validation 스타일 선호"].tolist(),
    "Validation_Dislike": summary_df["Validation 스타일 비선호"].tolist(),
}

In [43]:
user_data

{'R_id': [12,
  25,
  26,
  27,
  30,
  53,
  67,
  81,
  133,
  139,
  140,
  179,
  196,
  289,
  368,
  400,
  401,
  436,
  498,
  500,
  525,
  541,
  559,
  597,
  677,
  680,
  681,
  696,
  791,
  837,
  880,
  895,
  901,
  911,
  968,
  1004,
  1012,
  1022,
  1027,
  1047,
  1073,
  1105,
  1116,
  1174,
  1202,
  1229,
  1427,
  1511,
  1512,
  1513,
  1516,
  1609,
  1690,
  1880,
  1967,
  1968,
  1969,
  1970,
  1978,
  1987,
  1988,
  1990,
  1991,
  1993,
  1998,
  1999,
  2000,
  2002,
  2004,
  2007,
  2008,
  2011,
  2012,
  2013,
  2020,
  2026,
  2032,
  2042,
  2044,
  2045,
  2052,
  2060,
  2064,
  2065,
  2069,
  2072,
  2081,
  2085,
  2093,
  2102,
  2106,
  2122,
  2124,
  2125,
  2146,
  2177,
  2228,
  2245,
  2254,
  2265,
  2267,
  2309,
  2320,
  2337,
  2339,
  2386,
  2426,
  2442,
  2486,
  2525,
  2574,
  2609,
  2667,
  2672,
  2689,
  2740,
  2768,
  2817,
  3022,
  3076,
  3097,
  3248,
  3314,
  3439,
  3441,
  3445,
  3461,
  3540,
  3564,
  3

In [45]:
import json

# JSON 파일로 저장
with open("user_data.json", "w", encoding="utf-8") as file:
    json.dump(user_data, file, ensure_ascii=False, indent=4)  # indent=4는 보기 좋게 들여쓰기